# RunnableLambda : 사용자 정의 함수를 매핑하기

## 1. Set API Keys

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

## 2. RunnableLambda

`RunnableLambda`를 활용해서 프롬프트 입력으로 들어가기 전에 사용자 정의 함수를 통해 특정 처리를 마친 값을 넣어줄 수 있다.

In [9]:
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser
from datetime import datetime

# RunnableLambda로 감싸서 invoke시 무조건 입력값이 들어가는 구조가 되기 때문에 내부에서 입력값을 사용하지 않아도 더미 변수가 존재해야함
# 다만 RunnableLambda로 부터 넘어오는 파라미터는 1개이기 때문에, 파라미터가 2개로 구현된 함수는 RunnableLambda로 감쌀 수 없음
def get_today(tmp=None): 
    return datetime.today().strftime("%b-%d")

get_today()

'Feb-27'

In [7]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열해주세요. 생년월일도 함께 표기해주세요."
)

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

chain = (
    {"today" : RunnableLambda(get_today), "n" : RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke(3))

물론입니다! 2월 27일에 태어난 유명인 3명은 다음과 같습니다:

1. **헨리 포드 (Henry Ford)** - 1863년 2월 27일
2. **존 뉴먼 (John Steinbeck)** - 1902년 2월 27일
3. **엘리자베스 테일러 (Elizabeth Taylor)** - 1932년 2월 27일

이 외에도 많은 유명인들이 이 날에 태어났습니다!


Chain의 입력 값을 `3`이 아닌 `{"n" : 3} 과 같이 전달한 경우
- `"n" : RunnablePassthrough()` 이 부분이 `"n" : {"n" : 3}` 이렇게 처리되어 부정확한 결과를 초래할 수 있음
- 이럴 떄 사용하는 것이 `itemgetter()` 임

In [8]:
from operator import itemgetter

prompt = PromptTemplate.from_template(
    "{today}가 생일인 유명인 {n}명을 나열해주세요. 생년월일도 함께 표기해주세요."
)

model = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0.1
)

chain = (
    {"today" : RunnableLambda(get_today), "n" : itemgetter("n")}
    | prompt
    | model
    | StrOutputParser()
)

print(chain.invoke({"n" : 3}))

물론입니다! 2월 27일에 태어난 유명인 3명은 다음과 같습니다:

1. **헨리 포드 (Henry Ford)** - 1863년 2월 27일
2. **존 뉴먼 (John Steinbeck)** - 1902년 2월 27일
3. **버지니아 울프 (Virginia Woolf)** - 1882년 2월 27일

이 외에도 많은 유명인들이 이 날에 태어났습니다!
